In [2]:
import numpy as np
from hello.pid.lvpid import PIDController
from hello.pid.delay import seconds, minutes, hours, days, m2s, s2m, h2s, DelayBuffer, DelaySink
from hello.pid.gas_process import HeadspaceProcess, GasController
from hello.pid.doprocess import DOProcess, AIR_CNO

from hello.pid.plots import Data, SeriesData, PVPlot, GasesPlot, PIDPlot
from hello.pid.ui import EventDispatcher, TkQuitHack, MenuFrame, MenuLabel, Combobox, \
                        EntryButton, LabelDisplay, LabelDisplayFrame, LabeledEntry, \
                        LabelFrameWithLabelEntries

In [3]:
import json
from pysrc.snippets.option import OptionCategory

class PIDOps(OptionCategory):
    p = 5
    i = 5
    d = 0
    amax = 100
    amin = 0
    alpha = 1
    beta = 1
    linearity = 1
    gamma = 0
    deadband = 0
    man_request = 0
    mode = 0
    
class MFCOps(OptionCategory):
    co2_max = 1
    o2_max = 2
    n2_max = 10
    air_max = 10
    
class PlotOps(OptionCategory):
    xscale = 'auto'
    xmin = 0
    xmax = 72
    xscale_factor = 3600

class SimOps(OptionCategory):
    
    o2_pid = PIDOps()
    o2_pid.p = 2
    o2_pid.i = 10
    o2_pid.d = 0
    o2_pid.amax = 100
    o2_pid.amin = 0
    o2_pid.beta = 1
    o2_pid.linearity = 1
    o2_pid.alpha = -1
    
    n2_pid = PIDOps()
    n2_pid.p = 5
    n2_pid.i = 5
    n2_pid.d = 0
    n2_pid.amax = 90
    n2_pid.amin = 0
    n2_pid.beta = 1
    n2_pid.linearity = 1
    n2_pid.alpha = -1
    
    mfcs = MFCOps()
    mfcs.co2_max = 1
    mfcs.o2_max = 10
    mfcs.n2_max = 10
    mfcs.air_max = 10
    
    plots = PlotOps()
    plots.xscale = 'auto'
    plots.xmin = 0
    plots.xmax = 72
    plots.xscale_factor = 3600
    
    delay = 0
    end = 10000
    initial_actual_cno = None
    initial_request_cno = (0.07, 0, 0)
    initial_pv = 90
    set_point = 40
    set_point_deadband = 1
    k_mult = 1.1
    k = None
    c = None
    dc = 0
    d2c = 0
    mode = "o2a"
    main_gas = 1.0
    reactor_size = 80
    reactor_volume = 55
    time_unit = hours
    max_iters = 3 * days


### Copy this template to ensure all sim options are set correctly

In [ ]:
ops.o2_pid.p = 2
ops.o2_pid.i = 10
ops.o2_pid.d = 0
ops.o2_pid.amax = 100
ops.o2_pid.amin = 0
ops.o2_pid.beta = 1
ops.o2_pid.linearity = 1
ops.o2_pid.alpha = -1

ops.n2_pid.p = 5
ops.n2_pid.i = 5
ops.n2_pid.d = 0
ops.n2_pid.amax = 90
ops.n2_pid.amin = 0
ops.n2_pid.beta = 1
ops.n2_pid.linearity = 1
ops.n2_pid.alpha = -1

ops.mfcs.co2_max = 1
ops.mfcs.o2_max = 10
ops.mfcs.n2_max = 10
ops.mfcs.air_max = 10

ops.plots.xscale = 'auto'
ops.plots.xmin = 0
ops.plots.xmax = 72
ops.plots.xscale_factor = 3600

ops.delay = 0
ops.end = 10000
ops.initial_actual_cno = None
ops.initial_request_cno = (0.07, 0, 0)
ops.initial_pv = 90
ops.set_point = 40
ops.set_point_deadband = 1
ops.k_mult = 1.1
ops.k = None
ops.c = None
ops.dc = 0
ops.d2c = 0
ops.mode = "o2a"
ops.main_gas = 1.0
ops.reactor_size = 80
ops.reactor_volume = 55
ops.time_unit = hours
ops.max_iters = 3 * days

In [72]:
def _setpid(ob, state, name, value):
    pid = state[ob]
    if name == "pgain":
        pid.pgain = value
    elif name == "itime":
        pid.itime = m2s(value)
    elif name == "dtime":
        pid.dtime = m2s(value)
    elif name == "beta":
        pid.b = value
    elif name == "mode":
        sp = state['sp']
        if ob[:2] == "n2":
            sp += state['db']
        elif ob[:2] == 'o2':
            sp -= state['db']
        else:
            print("BAD PID")
        pid.set_mode(value, state['pv'], sp)
    elif name == "man":
        pid.man_request = int(value)
    else:
        print("WARNING: Invalid attribute for %r: %r" % (ob, name, name == "man"))

def _update_value(ops, state, proc, ob, name, value):
    if ob == "process":
        if name == "delay":
            value *= 60
        proc.set_values(**{name:value})
        return
    elif ob in ("o2_pid", "n2_pid"):
        _setpid(ob, state, name, value)
        return
    print("WARNING: Invalid attribute or object: %r %r" % (ob, name))

    
def _mk_pid(pidops, pv, sp, req, mode):
    c = PIDController(pgain=pidops.p,              itime=pidops.i,
                            dtime=pidops.d,             auto_max=pidops.amax,
                            auto_min=pidops.amin,       
                            beta=pidops.beta,
                            linearity=pidops.linearity, alpha=pidops.alpha,
                            deadband=pidops.deadband,   sp_high=100, sp_low=0,
                            gamma=pidops.gamma,         man_request=pidops.man_request,
                            mode=pidops.mode)
    if mode == "o2a":
        c.off_to_auto(pv, sp)
    elif mode == "m2a":
        c.man_to_auto(pv, sp, req)
    elif mode == "a2a":
        c.man_to_auto(pv, pv, req)
    else:
        raise ValueError(mode)
    return c
    

def do_sim_coroutine(ops, state, xq, pvq, cq,
                     co2q, n2q, o2q, aq, 
                     nukq, nupq, nuiq, nudq,
                     oukq, oupq, ouiq, oudq):
    
    """ This is the longest function in the history of man. 
    Its long becaues there are a *lot* of values to use and unpack for the inner PID loop,
    including code needed to synchronize with UI updates.
    Yay. 
    """
    
    co2_req, n2_req, o2_req = ops.initial_request_cno
    air_req = max(1-co2_req - n2_req - o2_req, 0)
    pv = ops.initial_pv
    sp = ops.set_point
    mode = ops.mode
    db = ops.set_point_deadband
    
    o2_pid = _mk_pid(ops.o2_pid, pv, sp-db, o2_req*100, mode)
    n2_pid = _mk_pid(ops.n2_pid, pv, sp+db, n2_req*100, mode)
    
    delay = ops.delay
    mg = ops.main_gas
    co2a, n2a, o2a = ops.initial_actual_cno
    
    proc = DOProcess(mg, pv,     (co2a, n2a, o2a), 
                     ops.reactor_size, ops.reactor_volume, 
                     ops.delay)
    
    proc.set_values(k=ops.k, k_mult=ops.k_mult, c=ops.c, dc=ops.dc, d2c=ops.d2c)
        
    ctrl = GasController(ops.mfcs.co2_max, ops.mfcs.n2_max, ops.mfcs.o2_max, ops.mfcs.air_max)
    
    t = 0
    time_unit = ops.time_unit
    mi = ops.max_iters
    msg = None

    while True:

        cmd, arg = yield msg
        msg = None
        
        state['c'] = proc.c * 3600
        state['dc'] = proc.dc * 3600 * 3600
        state['d2c'] = proc.d2c * 3600 * 3600 * 3600
        state['k'] = proc.k
        state['sp'] = sp
        state['db'] = db
        state['pv'] = pv
        state['mg'] = mg
        state['t'] = t
        state['co2_req'] = co2_req
        state['n2_req'] = n2_req
        state['o2_req'] = o2_req
        state['air_req'] = air_req
        state['o2_pid'] = o2_pid
        state['n2_pid'] = n2_pid

        if cmd == "SIM_ITERS":
            
            iters = arg
            if iters < 0:
                continue
            elif iters > mi:
                iters = mi

            # One approximation made in this simulation
            # is that no values, including oxygen consumption
            # (c), change during the inner iteration period 
            # outside of modification by the process or PID
            # classes themselves. 

            while iters > 0:

                t += 1
                iters -= 1
                o2_req = o2_pid.step(pv, sp-db) / 100
                n2_req = n2_pid.step(pv, sp+db) / 100
                co2_req, n2_req, o2_req, air_req = ctrl.request(mg, co2_req, n2_req, o2_req)
                pv = proc.step(pv, co2_req, n2_req, o2_req, air_req)

                xq.put(t/time_unit)
                pvq.put(pv)
                cq.put(proc.c * 3600)

                co2q.put(co2_req)
                n2q.put(n2_req)
                o2q.put(o2_req)
                aq.put(air_req)

                nukq.put(n2_pid.Uk)
                nupq.put(n2_pid.Up)
                nuiq.put(n2_pid.Ui)
                nudq.put(n2_pid.Ud)

                oukq.put(o2_pid.Uk)
                oupq.put(o2_pid.Up)
                ouiq.put(o2_pid.Ui)
                oudq.put(o2_pid.Ud)

        elif cmd == "SET_TIME":
            t = arg
        elif cmd == "UPDATE_VALUE":
            ob, name, value = arg
            rsp = _update_value(ops, state, proc, ob, name, value)
        elif cmd == "MODIFY_STATE":
            name, value = arg
            if name == "sp":
                sp = state['sp'] = value
            elif name == "pv":
                pv = state['pv'] = value
            elif name == "main_gas":
                mg = state['mg'] = value
            else:
                print("WARNING: Unrecognized name: %r" % name)
        else:
            raise ValueError(cmd)


In [5]:
def do_sim2(ops):
    # standalone sim function compatible with the do_sim_coroutine
    # function used for interactive use
    state = {}
    
    class MyList(list):
        # do_sim_coroutine requires the 
        # data containers have a "put" function
        put = list.append
    
    xq = MyList()
    pvq = MyList()
    cq = MyList()
    co2q = MyList()
    n2q = MyList()
    o2q = MyList()
    aq = MyList()
    nukq = MyList()
    nupq = MyList()
    nuiq = MyList()
    nudq = MyList()
    oukq = MyList()
    oupq = MyList()
    ouiq = MyList()
    oudq = MyList()
    coro = do_sim_coroutine(ops, state, xq, pvq, cq,
                     co2q, n2q, o2q, aq, 
                     nukq, nupq, nuiq, nudq,
                     oukq, oupq, ouiq, oudq)
    
    next(coro)
    coro.send(("SIM_ITERS", ops.end))
    
    data = list(zip(xq, pvq, cq,
                     co2q, n2q, o2q, aq))
    data2 = list(zip(nukq, nupq, nuiq, nudq))
    data3 = list(zip(oukq, oupq, ouiq, oudq))
    return data, data2, data3
    
def do_sim3(ops):
    # standalone sim function compatible with the do_sim_coroutine
    # function used for interactive use
    
    # This one yields the coroutine instead of calling it automatically
    # allowing for complex behavior made by adjusting parameters programmatically
    # mid-run. 
    state = {}
    
    class MyList(list):
        put = list.append
    
    xq = MyList()
    pvq = MyList()
    cq = MyList()
    co2q = MyList()
    n2q = MyList()
    o2q = MyList()
    aq = MyList()
    nukq = MyList()
    nupq = MyList()
    nuiq = MyList()
    nudq = MyList()
    oukq = MyList()
    oupq = MyList()
    ouiq = MyList()
    oudq = MyList()
    coro = do_sim_coroutine(ops, state, xq, pvq, cq,
                     co2q, n2q, o2q, aq, 
                     nukq, nupq, nuiq, nudq,
                     oukq, oupq, ouiq, oudq)
    
    next(coro)
    yield coro
    data = list(zip(xq, pvq, cq,
                     co2q, n2q, o2q, aq))
    data2 = list(zip(nukq, nupq, nuiq, nudq))
    data3 = list(zip(oukq, oupq, ouiq, oudq))
    yield data, data2, data3
    

In [6]:
# O2 needs an extra padding space because the negative sign
# for N2 pgain doesn't count, i guesss
def fmt_float(f):
    if not f:
        return "0"
    f = str(f)
    if "." not in f:
        return f
    else:
        f = f.strip("0")
        if f[-1] == ".":
            f = f[:-1]
        if f[0] == ".":
            f = "0" + f
    return f
def get_text(ops):

    text = \
"""
O2: P=%3d  I=%5d  Initial PV: %3d  k: %.4f/hr (%sx Est.)
N2: P=%3d  I=%4d  Set Point:  %3d  c: %s%%/hr

 """ % (ops.o2_pid.p, ops.o2_pid.i,ops.initial_pv, ops.k*ops.k_mult, fmt_float(ops.k_mult),
        ops.n2_pid.p, ops.n2_pid.i, ops.set_point, fmt_float(ops.c))
    text = text.strip()
    return text

In [7]:
def run(ops):
    global data, data2, text, data3
    data, data2, data3 = do_sim2(ops)
    text = get_text(ops)
    plot(ops)

In [8]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib

def plot(ops):    
    print("Parsing Data")
    global x, pv, mg, cs, co2_req, n2_req, o2_req, air_req, co2a, n2a, o2a, uk, up, ui, ud
    x, pv, cs, co2_req, n2_req, o2_req, air_req = list(zip(*data))
    uk, up, ui, ud = list(zip(*data2))
    print("Closing Plot")
    plt.close()
    print("Plotting Data")

    step = 100
    xs = x[::step]
    xs = np.array(xs) / ops.plots.xscale_factor * ops.time_unit
    fig = plt.figure()
    ax1 = plt.subplot(311)
    ax2 = plt.subplot(312)
    ax3 = plt.subplot(313)

    pvs = np.array(pv[::step])
    ax1.plot(xs, pvs, "blue", ls="-", label="PV")
    ax1.axhline(y=sp, ls="--", color="black")
    # abs_err = np.abs(sp - pvs)
    # esq **= 2
    # esq = np.sqrt(esq)
    #ax1.plot(xs, abs_err, "red", ls="-", label="Err^2")

    #ax2.plot(xs, co2_req[::step], "purple", ls="-", label="CO2")
    ax2.plot(xs, n2_req[::step], "red", ls="-", label="N2")
    ax2.plot(xs, o2_req[::step], "green", ls="-", label="O2")
    ax2.plot(xs, air_req[::step], "cyan", ls="-", label="Air")
    fm1 = FuncFormatter(lambda y, _: "%.2f%%"%y)
    fm2 = FuncFormatter(lambda y, _: "%.2f%%"%(y*100))

    ax3.plot(xs, uk[::step], "blue", ls="-", label="Uk")
    ax3.plot(xs, up[::step], "red", ls="-", label="Up")
    ax3.plot(xs, ui[::step], "green", ls="-", label="Ui")
    ax3.plot(xs, ud[::step], "purple", ls="-", label="Ud")

    ax1.yaxis.set_major_formatter(fm1)
    ax2.yaxis.set_major_formatter(fm2)
    ax2.set_ylim((0, 1.1))
    
    m, ma = ax1.get_ylim()
    if np.min(pvs) - 1 < m:
        m -= 1
    if np.max(pvs) + 1 > ma:
        ma += 1
    ax1.set_ylim((m, ma))
    #ax1.set_ylim((sp-5, sp+5))


    for a in ax1, ax2, ax3: 
        b = a.get_position()
        a.set_position([b.x0, b.y0, b.width*0.9, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.grid()
        if ops.plots.xscale == 'man':
            a.set_xlim((ops.plots.xmin, ops.plots.xmax))
    
    fig.text(0.15, 0.95, get_text(ops), transform=ax1.transAxes, fontsize=12,
        verticalalignment='top')
    
    wm=plt.get_current_fig_manager()
    wm.window.attributes('-topmost', 1)
    wm.window.attributes('-topmost', 0)
    # h = wm.window.winfo_height()
    # w = wm.window.winfo_width()
    wm.window.geometry("%sx%s+%s+%s"%(700,720,50, 20))

Using matplotlib backend: TkAgg


In [9]:
ops = SimOps()
ops.end = 10 * hours
sp = 80

ops.o2_pid.p = 10
ops.o2_pid.i = 80
ops.o2_pid.amax = 100
ops.o2_pid.deadband = 0
ops.o2_pid.d = 0
ops.o2_pid.alpha = -1

ops.n2_pid.p = -2
ops.n2_pid.i = 60
ops.n2_pid.amax = 100
ops.n2_pid.deadband = 0
ops.n2_pid.d = 0
ops.n2_pid.alpha = -1

ops.initial_pv = 20
ops.set_point = sp
ops.c = 0
ops.dc = 0
ops.d2c = 0

# k_mult = 1.25
ops.k_mult = 1
ops.k = 0.19314
ops.initial_actual_cno = (0.0000, .90, .1)
ops.initial_request_cno = (0.00, .78, 0)
ops.mode = 'm2a'
ops.plots.xscale = 'auto'
ops.plots.xmin = 20
ops.plots.xmax = 30
ops.set_point_deadband = 1
ops.delay = 0

run(ops)

from matplotlib.ticker import MultipleLocator
l = MultipleLocator(1)
for a in plt.gcf().axes:
    a.xaxis.set_major_locator(l)

Parsing Data
Closing Plot
Plotting Data


In [210]:
class Fitness2():
    stable_osc_max = 1
    settle_osc_max = 3
    settle_target = 3 * hours
    max_overshoot = 3
    def __init__(self, data=None):
        if data is not None:
            self.calculate(data)
            
    def calculate(self, data):
        x,y,sp = data
        self.settle_time = self.time_stable(x,y,sp)
        self.osc = self.oscillation(x,y,sp)
        self.ovr = np.max(y) - sp
        
    def result_reason(self):
        rv = True
        reason = ""
        if self.settle_time > self.settle_target:
            reason += "Settle %d > %d " % (self.settle_time, self.settle_target)
            rv = False
        if self.osc > self.stable_osc_max:
            reason += "Osc %d > %d " % (int(sef.osc), self.stable_osc_max)
            rv = False
        if self.ovr > self.max_overshoot:
            reason += "Ovr %d > %d " % (int(self.ovr), int(self.max_overshoot))
            rv = False
        return rv, reason
    
    def result(self):
        return self.result_reason()[0]
    
    def result2(self):
        s = self.settle_time - self.settle_target
        o = self.stable_osc_max - self.osc
        return s, o
    
    def reason(self):
        return self.result_reason()[1]
        
    def oscillation(self, x, y, sp):
        span = y[10*hours:]
        if not len(span):
            return -1
        ymin = np.min(span)
        ymax = np.max(span)
        return (ymin - ymax) / 2
        
    def time_stable(self, x, y, setpoint):
        hi = setpoint + self.settle_osc_max
        lo = setpoint - self.settle_osc_max
        for t, pv in reversed(list(zip(x,y))):
            if not lo <= pv <= hi:
                break
        return t * 3600 + 1

In [208]:
import weakref

def damping_ratio(p1, p2, n=1):
    ln = np.log(p1 / p2)
    delta = (1/n) * ln
    z = 1 / (np.sqrt(1+(2*np.pi/delta)**2))
    return z
    
def dr2(ind):
    if not ind.size:
        return 0
    if ind.size == 1:
        return ind[0]
    ratios = []
    for i in range(len(ind)-1):
        ratios.append(damping_ratio(ind[i], ind[i+1]))
    return sum(ratios)/len(ratios)

def time_between(ind):
    if not ind.size:
        return -1
    return np.average(ind[1:]-ind[:-1])

def peaks(x, pv, sp=150, band=5, thresh=0.1):
    hi = sp + band
    lo = sp - band
    t = next((t for t, p in reversed(list(zip(x,pv))) if not lo <= p <= hi),0)
    t = int(t*3600)
    return _peaks(pv, t, len(x), thresh)

def _peaks(x, start, end, thresh=0.5):
    return peakutils.indexes(x[start:end], thres=thresh,min_dist=5*minutes) + start

_g_leg_map = weakref.WeakKeyDictionary()
_cb_id = 0
def on_legend_clicked2(fig, leg_map, event):
    ll = event.artist
    for line in leg_map[ll]:
        line.set_visible(not line.get_visible())
    fig.canvas.draw()

def mk_picker(fig, ax, othermap=None, new=True):
    othermap = othermap or {}
    global _g_leg_map, _cb_id
    _g_leg_map[fig] = _g_leg_map.get(fig, {})
    leg_map = _g_leg_map[fig]
    a2l = {}
    def _olc(e):
        on_legend_clicked2(fig, leg_map, e)
    
    for lline in ax.legend_.get_lines():
        for aline in ax.get_lines():
            if lline.get_label() == aline.get_label():
                leg_map[lline] = leg_map.get(lline, [])
                leg_map[lline].append(aline)
                lline.set_picker(5)
                aline.set_visible(True)
                a2l[aline] = lline
    for aline, others in othermap.items():
        lline = a2l[aline]
        leg_map[lline].extend(others)
    fig.canvas.mpl_disconnect(_cb_id)
    _cb_id = fig.canvas.mpl_connect("pick_event", _olc)

In [197]:
%matplotlib
ops.end = 10 * hours
sp = 150

ops.o2_pid.p = 1
ops.o2_pid.i = 50
ops.o2_pid.amax = 100
ops.o2_pid.deadband = 0
ops.o2_pid.d = 0
ops.o2_pid.alpha = -1

ops.n2_pid.p = -0.5
ops.n2_pid.i = 60
ops.n2_pid.amax = 100
ops.n2_pid.deadband = 0
ops.n2_pid.d = 0
ops.n2_pid.alpha = -1

ops.initial_pv = 100
ops.set_point = sp
ops.c = 0
ops.dc = 0
ops.d2c = 0

# k_mult = 1.25
ops.k_mult = 1
ops.k = 0.19314
ops.initial_actual_cno = (0.0000, .78, .2095)
ops.initial_request_cno = (0.00, 0, 0)
ops.mode = 'm2a'
ops.plots.xscale = 'auto'
ops.plots.xmin = 20
ops.plots.xmax = 30
ops.set_point_deadband = 1
ops.delay = 0

results = []
f = FuncFormatter(lambda y, _: "%.1f%%"%y)
plt.close()
fig = plt.figure()
ax = fig.add_subplot(3,1,1)
ax2 = fig.add_subplot(3,1,2)
ax3 = fig.add_subplot(3,1,3)
ax.axhline(y=sp, ls="--", color="black")
ax.grid()

ax.yaxis.set_major_formatter(f)
ax2.yaxis.set_major_formatter(f)
ax3.yaxis.set_major_formatter(f)

def plot2(x, pv, o2_req, n2_req, label):
    ax.plot(x, pv, label=label)
    ax2.plot(x, np.array(o2_req)*100, label=label)
    ax3.plot(x, np.array(n2_req)*100, label=label)

combos = [(p,i) for p in range(1,2) for i in range(50, 101, 10)]
    
for p, i in combos:
    if p is not None:
        ops.o2_pid.p = p
    if i is not None:
        ops.o2_pid.i = i
    d1, d2, d3 = do_sim2(ops)
    x, pv, cs, co2_req, n2_req, o2_req, air_req = list(zip(*d1))
    plot2(x, pv, o2_req, n2_req, "P:%.1f I:%.1f"%(p,i))
    
    x = np.array(x)
    pv = np.array(pv)
    ind = peakutils.indexes(pv, thres=0.5, min_dist = 5*minutes)
    ax.plot(x[ind], pv[ind], '+', mfc=None, mec='r', mew=2, ms=8)
    print(p,i, str(Fitness3(x,pv,150)), ind/3600, pv[ind])

for a in (ax, ax2, ax3):
    b = a.get_position()
    a.set_position([b.x0, b.y0, b.width*0.9, b.height])
    a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left", fontsize=10)
    mk_picker(fig, a)

Using matplotlib backend: TkAgg
1 50 Total:28.20 Peak:1.69 Settle:5.03 [ 3.71222222] [ 157.38968242]
1 60 Total:29.57 Peak:0.76 Settle:5.38 [ 4.19111111] [ 155.51376397]
1 70 Total:31.08 Peak:0.03 Settle:5.57 [ 4.68388889] [ 154.06694337]
1 80 Total:12.59 Peak:-0.54 Settle:3.51 [ 5.20305556] [ 152.92963296]
1 90 Total:16.31 Peak:-0.99 Settle:3.92 [ 5.76222222] [ 152.02617068]
1 100 Total:20.73 Peak:-1.35 Settle:4.35 [ 6.37888889] [ 151.30538625]


In [198]:
def sln_space_test(ops):
    d1, d2, d3 = do_sim2(ops)
    x, pv, cs, co2_req, n2_req, o2_req, air_req = [np.array(d) for d in zip(*d1)]
    cs = cs
    co2_req = co2_req
    n2_req = n2_req
    o2_req = o2_req
    air_req = air_req
    del d1, d2, d3  # garbage collect
    score = Fitness3(x, pv, ops.set_point)
    score2 = Fitness2((x, pv, ops.set_point))
    return locals().copy()

In [219]:
from mpl_toolkits.mplot3d import Axes3D
leg_map = {}

sp = 150

try:
    plot_test_results
except NameError:
    plot_test_results = {}

def on_legend_clicked(event):
    ll = event.artist
    line = leg_map[ll]
    line.set_visible(not line.get_visible())
    fig.canvas.draw()
fig.canvas.mpl_connect("pick_event", on_legend_clicked)


def plot2(x, pv, o2_req, n2_req, label, update=True):
    l1 = ax.plot(x, pv, label=label)
    ax.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left", fontsize=10)
    if update:
        fig.canvas.draw()
        fig.canvas.flush_events()
    
scatter_data = [[],[],[]]
def plot3(p, i, score, update=True):
    x,y,z = scatter_data
    x.append(p)
    y.append(i)
    z.append(score)
    ax3d.clear()
    ax3d.scatter(x,y,z, antialiased=False)
    if update:
        fig.canvas.draw()
        fig.canvas.flush_events()

plt.close()
fig = plt.figure()
ax = fig.add_subplot(2,1,1)
ax3d = fig.add_subplot(2,1,2, projection='3d')
ax.axhline(y=sp, ls="--", color="black")
ax.grid()
ax.axhline(y=sp+3, color="black", ls="--")
ax.axhline(y=sp-3, color="black", ls="--")
ax.set_ylim(sp-10, sp+10)

f = FuncFormatter(lambda y, _: "%.1f%%"%y)
ax.yaxis.set_major_formatter(f)

ops.o2_pid.p = 7
ops.o2_pid.i = 50
ops.o2_pid.d = 0
ops.o2_pid.amax = 100
ops.o2_pid.amin = 0
ops.o2_pid.beta = 0
ops.o2_pid.linearity = 1
ops.o2_pid.alpha = -1

ops.n2_pid.p = -6
ops.n2_pid.i = 90
ops.n2_pid.d = 0
ops.n2_pid.amax = 90
ops.n2_pid.amin = 0
ops.n2_pid.beta=0
ops.n2_pid.linearity = 1
ops.n2_pid.alpha = -1

ops.mfcs.co2_max = 0.1
ops.mfcs.n2_max = 2
ops.mfcs.o2_max = 0.5
ops.mfcs.air_max = 2

ops.initial_pv = 100
ops.set_point = sp
ops.mode="o2a"

ops.end = 10*hours
ops.set_point_deadband = 1
ops.delay = 0
ops.initial_actual_cno = AIR_CNO
ops.initial_request_cno = (0, 0, 0)
ops.k_mult = 1
ops.k = 0.19314
ops.reactor_size = 15
ops.reactor_volume = 11.5

ops.dc = 0
ops.d2c = 0

ops.main_gas = 1.0
ops.time_unit = hours
ops.max_iters = 24 * days

axes = [ax, ax3d]

for a in axes:
    b = a.get_position()
    a.set_position([b.x0, b.y0, b.width*0.9, b.height])

passing = []
update = True
noupdate = 0
Fitness2.max_overshoot = 2
Fitness2.settle_target = 2 * hours
for p in range(6, 9):
    for i in range(50, 81, 10):
        print("\rTesting P:%d I:%d " % (p,i), end="")
        if (p,i) not in plot_test_results:
            ops.o2_pid.p = p
            ops.o2_pid.i = i
            data = sln_space_test(ops)
            update = True
        else:
            data = plot_test_results[(p,i)]
            update = False
            noupdate += 1
            if noupdate > 10:
                noupdate = 0
                update = True
        s2 = Fitness2((data['x'], data['pv'], sp))
        res, reason = s2.result_reason()
        if True == True:
            passing.append((p,i))
            label = "P:%3d I:%3d" % (p, i)
            plot2(data['x'][::120], data['pv'][::120],data['o2_req'], data['n2_req'], label, update)
            plot_test_results[(p,i)] = data
        plot3(p,i, data['score'].score(), update)
        print("Result: %s %s %s" % (str(data['score']), res, reason))
        fig.canvas.flush_events()
print("Done")

Testing P:6 I:50 Result: Total:2.80 Peak:-1.09 Settle:2.00 True 
Testing P:6 I:60 Result: Total:3.84 Peak:-2.00 Settle:2.80 False Settle 10080 > 7200 
Testing P:6 I:70 Result: Total:8.19 Peak:-2.00 Settle:3.49 False Settle 12569 > 7200 
Testing P:6 I:80 Result: Total:11.51 Peak:-2.00 Settle:3.94 False Settle 14180 > 7200 
Testing P:7 I:50 Result: Total:0.93 Peak:-1.73 Settle:1.98 True 
Testing P:7 I:60 Result: Total:3.78 Peak:-2.00 Settle:2.79 False Settle 10042 > 7200 
Testing P:7 I:70 Result: Total:8.31 Peak:-2.00 Settle:3.51 False Settle 12631 > 7200 
Testing P:7 I:80 Result: Total:11.78 Peak:-2.00 Settle:3.97 False Settle 14299 > 7200 
Testing P:8 I:50 Result: Total:4.75 Peak:0.92 Settle:1.98 True 
Testing P:8 I:60 Result: Total:3.71 Peak:-2.00 Settle:2.78 False Settle 9997 > 7200 
Testing P:8 I:70 Result: Total:8.40 Peak:-2.00 Settle:3.52 False Settle 12678 > 7200 
Testing P:8 I:80 Result: Total:11.98 Peak:-2.00 Settle:4.00 False Settle 14393 > 7200 
Done


In [220]:
mk_picker(fig, ax)

In [202]:
class Fitness3():
    settle_factor = 1 * hours
    peak_factor = 2 * hours
    ramp_factor = 1.5 * hours
    settle_margin = 3
    damp_factor = 0.1
    
    # The normalizing scale for dampening factor
    # uses an equation of the form a/(b+x*c)-d
    a = 4
    b = 1
    c = 7
    d = 2
    
    def __init__(self, x, y, sp):
        st = self.margin(x,y,sp,self.settle_margin)
        sc = st / self.settle_factor
        self.settle_score = sc
        
        if st >= len(y):
            # no peaks
            self.peak_score = 0
        else:
            ind = _peaks(y, 0, len(y), 0.5)
            if not ind.size:
                ps=-2
            else:
                dr = dr2(ind)
                ps = self.a/(self.b+dr*self.c) - self.d
            mscore = (np.max(y)-sp)/2
            mscore = min(max(0, mscore),5)
            self.peak_score = mscore + ps
        
    def score(self):
        return self.settle_score**2*np.sign(self.settle_score) + \
                self.peak_score**2*np.sign(self.peak_score)
        
    def margin(self, x, y, setpoint, margin):
        hi = setpoint + margin
        lo = setpoint - margin
        for t, pv in reversed(list(zip(x,y))):
            if not lo <= pv <= hi:
                break
        return (t) * 3600 + 1
    
    def __str__(self):
        return "Total:%.2f Peak:%.2f Settle:%.2f" % (self.score(), self.peak_score, self.settle_score)

In [125]:
def plot4(pd, sp):
    global fig, ax, ax2, ax3
    _g_leg_map.clear()
    mp = {}
    if not plt.get_fignums():
        fig = plt.figure()
        new = True
        ax = fig.add_subplot(311)
        ax2 = fig.add_subplot(3,1,2)
        ax3 = fig.add_subplot(3,1,3)
    else:
        new = False
        ax.clear()
        ax2.clear()
        ax3.clear()

    f = FuncFormatter(lambda y, _: "%.1f%%"%(y*100))
    ax2.yaxis.set_major_formatter(f)
    ax3.yaxis.set_major_formatter(f)
    print(*("%-12s"%a for a in "p i settle osc".split()))
    def _p(*args):
        print(*("%-12.1f"%a for a in args))
    for (p,i), (x,pv,n2,o2) in pd.items():
        f =  Fitness2()
        f.settle_target = 5 * hours
        f.calculate((x,pv,50))
        s,o = f.result2()
        _p(p,i, s/3600, o)
        label = "P:%d I:%d"%(p,i)
        ax.plot(x,pv, label=label + " PV")
        ax2.plot(x, n2, label=label+" N2")
        ax3.plot(x, o2, label=label+" O2")
    for a in (ax, ax2, ax3):
        if new:
            b = a.get_position()
            a.set_position([b.x0, b.y0, b.width*0.80, b.height])
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        mk_picker(fig, a)
    ax.axhline(y=sp-1, ls="--", color="black")
    ax.axhline(y=sp+1, ls="--", color="black")
    ax.set_ylim(sp-5, sp+5)
    #ax2.set_ylim(0, 110)
    
def test3(ops):
    d1, d2, d3 = do_sim2(ops)
    x, pv, _, _, n2, o2, *_ = list(zip(*d1))
    return x,pv, n2, o2
    

In [16]:
# O2 PID
pd = {}
passing = [
    (6, 60),
    (7, 50),
    (8, 50),
    (9, 50),
    (10, 50)
]
sp = 150
ops.end = 20*hours
ops.initial_pv = 100
ops.set_point = sp
for p, i in passing:
    print("Testing P:%d I:%d"%(p,i))
    ops.o2_pid.p = p
    ops.o2_pid.i = i
#     ops.n2_pid.p = -8
#     ops.n2_pid.i = 100
    ops.n2_pid.p = -2
#    ops.n2_pid.i = 60
    pd[(p,i)] = test(ops)

Testing P:6 I:60
Testing P:7 I:50
Testing P:8 I:50
Testing P:9 I:50
Testing P:10 I:50


In [17]:
# N2 PID
pd = {}
ps = (-6, -8, -10)
its = (80, 90, 100)
params = [(p, i) for p in ps for i in its]
ops.end = 20*hours
ops.initial_pv = 100
sp = 50
ops.set_point = sp
for p, i in params:
    print("Testing P:%d I:%d"%(p,i))
    ops.o2_pid.p = 7
    ops.o2_pid.i = 50
    ops.n2_pid.p = p
    ops.n2_pid.i = i
    pd[(p,i)] = test(ops)

Testing P:-6 I:80
Testing P:-6 I:90
Testing P:-6 I:100
Testing P:-8 I:80
Testing P:-8 I:90
Testing P:-8 I:100
Testing P:-10 I:80
Testing P:-10 I:90
Testing P:-10 I:100


In [18]:
plot4(pd, sp)

p            i            settle       osc         
-10.0        100.0        0.5          1.1         
-10.0        90.0         -0.3         1.0         
-6.0         100.0        -0.4         1.1         
-6.0         90.0         -0.6         1.1         
-6.0         80.0         -0.7         1.1         
-10.0        80.0         -0.6         1.0         
-8.0         80.0         -0.7         1.1         
-8.0         90.0         -0.5         1.1         
-8.0         100.0        -0.2         1.0         


In [ ]:
pd = {}
sp = 150
p,i = -2, 60
ops.o2_pid.p = 7
ops.o2_pid.i = 50
ops.n2_pid.p = p
ops.n2_pid.i = i
ops.set_point = sp
pd[(p,i)] = test(ops)
p,i = -0.5, 60
ops.n2_pid.p = p
ops.n2_pid.i = i
pd[(p,i)] = test(ops)
plot4(pd, sp)

In [94]:
sp=100
ops.o2_pid.p = 7
ops.o2_pid.i = 50
ops.n2_pid.p = -6
ops.n2_pid.i = 90
ops.n2_pid.amax = 90
ops.n2_pid.beta=0
ops.o2_pid.beta = 0
ops.initial_pv = 50
ops.set_point = sp
ops.mfcs.air_max = 2
ops.mfcs.n2_max = 2
ops.mfcs.co2_max = 0.1
ops.mfcs.o2_max = 0.5
n2 = 0.49
ops.initial_actual_cno = HeadspaceProcess.actual_from_request(0, n2, 0, 1-n2)[:3]
ops.initial_request_cno = (0,n2,0)
ops.mode = "a2a"
#ops.k_mult = 0.9
pd = {}
for km in (0.9, 1.0, 1.1):
    ops.k_mult = 1.0
    data = test(ops)
    pd[(-6, 90)] = data
    break
plot4(pd, sp)

from matplotlib.ticker import MultipleLocator
for a in (ax, ax2, ax3):
    m = MultipleLocator(2)
    a.xaxis.set_major_locator(m)
    a.grid()

p            i            settle       osc         
-6.0         90.0         15.0         1.3         


# Begin 3/10/17 Excel Data Comparison

In [19]:
from officelib.xllib import *
xl = Excel()
for wb in xl.Workbooks:
    print(wb.Name)

In [20]:
wb = xl.RecentFiles(1).Open()
cells = wb.Worksheets("Sheet1").Cells
cell_range = cells.Range

In [21]:
from pysrc.snippets import smooth1

In [23]:
def xlimport(r):
    rng = cell_range(r)
    data = rng.Value
    x,y=list(zip(*data))
    sx, sy = smooth1(x,y)
    sy = [float(f) for f in sy]
    assert sx[:10] == list(range(10))
    return sy

In [109]:
n2a = xlimport("B2:C1043")
o2a = xlimport("J2:K1385")
dopv = xlimport("N2:O1018")
n2a=np.array(n2a)/100
o2a=np.array(o2a)/100
data3 = list(zip(np.arange(min(len(l) for l in (dopv, n2a, o2a)))/3600, dopv, n2a, o2a))

## Basic data comparison

In [112]:
def test2(ops):
    pv = 51
    sp = 100
    db = 1
    o2_req = 0
    n2_req = 0.509
    co2_req = 0

    sp=100
    ops.o2_pid.p = 7
    ops.o2_pid.i = 50
    ops.n2_pid.p = -6
    ops.n2_pid.i = 90
    ops.n2_pid.amax = 90
    ops.n2_pid.beta=0
    ops.o2_pid.beta = 0
    ops.initial_pv = pv
    ops.set_point = sp
    ops.mode="a2a"
    ops.initial_actual_cno = HeadspaceProcess.actual_from_request(0, n2_req, 0, 1-n2_req)[:3]
    ops.initial_request_cno = (0, n2_req, 0)
    ops.mfcs.co2_max = 0.1
    ops.mfcs.n2_max = 2
    ops.mfcs.o2_max = 0.5
    ops.mfcs.air_max = 2
    ops.k_mult=1
    ops.reactor_size = 15
    ops.reactor_volume = 11.5

    o2_pid = _mk_pid(ops.o2_pid, pv, sp-db, o2_req*100, "a2a")
    n2_pid = _mk_pid(ops.n2_pid, pv, sp+db, n2_req*100, "a2a")

    ctrl = GasController(ops.mfcs.co2_max, ops.mfcs.n2_max, ops.mfcs.o2_max, ops.mfcs.air_max)
    mg = 1.0
    data = []
    for i, pv in enumerate(dopv):
        o2_req = o2_pid.step(pv, sp-db) / 100
        n2_req = n2_pid.step(pv, sp+db) / 100
        co2_req, n2_req, o2_req, air_req = ctrl.request(mg, co2_req, n2_req, o2_req)
        data.append((i/3600, pv, n2_req, o2_req))
    return data


In [101]:
from matplotlib.ticker import FuncFormatter, FormatStrFormatter, MultipleLocator

def axfmt(a):
    b = a.get_position()
    a.set_position([b.x0, b.y0, b.width*0.8, b.height])

if not plt.get_fignums():
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax2 = ax.twinx()
    for a in (ax, ax2): axfmt(a)
else:
    ax.clear()
    ax2.clear()

def plot5(data, ax, ax2, olbl=""):
    ax, ax2 = ax2, ax
    x, pv, n2_req, o2_req = list(zip(*data))
    f1 = FuncFormatter(lambda y, _: "%.1f%%"%(y*100))
    f2 = FormatStrFormatter("%.1f%%")
    ax.yaxis.set_major_formatter(f2)
    ax2.yaxis.set_major_formatter(f1)
    l = MultipleLocator(2)
    ax.xaxis.set_major_locator(l)
    ax.plot(x, pv, color="blue", label=olbl+"PV")
    ax2.plot(x, o2_req, color="green", label=olbl+"O2")
    ax2.plot(x, n2_req, color="red", label=olbl+"N2")
    h1,l1 = ax.get_legend_handles_labels()
    h2,l2 = ax2.get_legend_handles_labels()
    h1.extend(h2)
    l1.extend(l2)
    ax.legend(h1, l1, bbox_to_anchor=(1.12, 1.06), loc="upper left")
    ax2.grid()
    ax.grid()
    ax.set_xlim(0, 10)
    ax2.set_ylim(.0,1)
    ax.set_ylim(80, 110)
#plot5(data,ax,ax2)

In [98]:
#def man_to_auto(pv, sp, op, b, pgain):
    err_for_pgain = (b*sp-pv)*(1+(1-1)*(abs(b*sp-pv))/100)
    uk0 = pgain * err_for_pgain
    Ui = op - uk0
    last_pv = pv
    last_err = 0
    last_ierr = 0
    last_berr = b * sp - pv
    last_output = op
    for k, v in locals().items():
        print(k,":", v)
    print("Exp OP:", uk0+Ui)
    
def man_to_auto2(self, pv, sp, op):
    err_for_pgain = (self.b*sp-pv)# *(self.l+(1-self.l)*(abs(self.b*sp-pv))/100)
    err = sp - pv
    Up = self.pgain * err_for_pgain
    self.Up = Up
    self.Ui = op - Up
    self.last_pv = pv
    self.last_err = err
    self.last_ierr = err*self.pgain*self.oneoveritime
    self.last_gerr = self.g * sp - pv
    self.last_berr = self.b * sp - pv
    self.last_output = op
    for k, v in locals().items():
        print(k,":", v)
    print("Exp OP:", Up+self.Ui)
    
def step(self, pv, sp):
    import pdb
    pdb.set_trace()
    if self._mode != 0:
        if self._mode == 1:
            return self.man_request
        elif self._mode == 2:
            return 0
        else:
            raise ValueError(self._mode)
    err = sp - pv
    berr = self.b * sp - pv
    gerr = self.g * sp - pv

    # Based on Labview's `PID Get Advanced Error (DBL).vi`
    if self.l == 1 or self.sp_high == self.sp_low:
        nl_int_factor = 1
    else:
        sp_rng = self.sp_high - self.sp_low
        oml = 1 - self.l
        nl_int_factor = 1 / (1+err*err/ (sp_rng*sp_rng))
        err = err * (self.l + oml*abs(err)/sp_rng)
        berr = berr * (self.l + oml*abs(berr)/sp_rng)
        gerr = gerr * (self.l + oml*abs(gerr)/sp_rng)

    errs_avg = (err + self.last_err) / 2
    ierr = errs_avg * nl_int_factor * self.pgain * self.oneoveritime

    
    Up = self.pgain * berr
    Ui = self.Ui + ierr
    if 0 < self.a <= 1:
        Ud = (self.Ud - ((self.pgain / self.a) * (gerr - self.last_gerr))) * (self.dtime / (self.dtime + (1 / self.a)))
    else:
        Ud = -(self.last_gerr - gerr) * self.pgain * self.dtime 
    Uk = Up + Ui + Ud

    # Coercion & back calculation      
    if Uk > self.auto_max:
        Uk = self.auto_max
        Ui = Uk - Up - Ud
    elif Uk < self.auto_min:
        Uk = self.auto_min
        Ui = Uk - Up - Ud

    # XXX debugging
    self.Ui = Ui
    self.Uk = Uk
    self.Ud = Ud
    self.Up = Up

    # "Shift Register"
    self.last_output = Uk
    self.last_pv = pv
    self.last_err = err
    self.last_ierr = ierr
    self.last_gerr = gerr

    return Uk

def _mk_pid(pidops, pv, sp, req, mode):
    c = PIDController(pgain=pidops.p,              itime=pidops.i,
                            dtime=pidops.d,             auto_max=pidops.amax,
                            auto_min=pidops.amin,       
                            beta=pidops.beta,
                            linearity=pidops.linearity, alpha=pidops.alpha,
                            deadband=pidops.deadband,   sp_high=100, sp_low=0,
                            gamma=pidops.gamma,         man_request=pidops.man_request,
                            mode=pidops.mode)
    if mode == "o2a":
        c.off_to_auto(pv, sp)
    elif mode == "m2a":
        c.man_to_auto(pv, sp, req)
    elif mode == "a2a":
        c.man_to_auto(pv, pv, req)
    else:
        raise ValueError(mode)
    return c
n2_pid = _mk_pid(ops.n2_pid, 51, 100+db, 50.9, "a2a")
n2_pid.b = 0
#man_to_auto(51, 100+1, 50.9, 0, -6)
#man_to_auto2(n2_pid, 51, 101, 50.9)
# print(n2_pid.Up, n2_pid.Ui)
# print(n2_pid.Ui,step(n2_pid, 51, 101))
# print("Uk", n2_pid.Uk)
# print("Up", n2_pid.Up)
# print("Ui", n2_pid.Ui)
n2_pid.step(51, 101)

IndentationError: unexpected indent (<ipython-input-98-97b0cfc21199>, line 2)

In [122]:
ops.k_mult=1
ops.reactor_size = 15
ops.reactor_volume = 11.5
data2 = list(zip(*test(ops)))

In [123]:
if not plt.get_fignums():
    fig = plt.figure()
    ax  = fig.add_subplot(311)
    ax2 = ax.twinx()
    ax3 = fig.add_subplot(313)
    ax4 = ax3.twinx()
    ax5 = fig.add_subplot(3,1,2)
    ax6 = ax5.twinx()
    for a in (ax, ax2, ax3, ax4, ax5, ax6):
        axfmt(a)
else:
    ax.clear()
    ax2.clear()
    ax3.clear()
    ax4.clear()
    ax5.clear()
    ax6.clear()
plot5(data,ax,ax2,"SimOP ")
plot5(data2, ax3, ax4, "SimALL ")
plot5(data3, ax5, ax6, "Real ")

In [121]:
from time import sleep
for k_mult in (1,):
    ops.k_mult = k_mult
    data2 = list(zip(*test(ops)))
    ax.clear()
    ax2.clear()
    ax3.clear()
    ax4.clear()
    ax5.clear()
    ax6.clear()
    plot5(data,ax,ax2,"SimOP ")
    plot5(data2, ax3, ax4, "SimALL ")
    plot5(data3, ax5, ax6, "Real ")
    fig.canvas.flush_events()
    fig.canvas.draw()
    sleep(3)
    

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.arange(len(n2a))/3600, n2a)

In [ ]:
for a in (ax, ax2, ax3, ax4, ax5, ax6):
    a.set_xlim(0,1)

# End 3/10/17 Excel Data Comparison

In [ ]:
plot(ops)

In [ ]:
# Maximum value of c that O2 controller can sustain for a given k
ops.k*ops.k_mult*(100/20.95)*100

In [ ]:
class SimConfig(OptionCategory):
    simops = ops or SimOps()
    xwindow_hrs = 20
    pause = False
    update_interval = 100
    time_factor = 200
    time_unit = 3600
    max_iters = hours * 24 * 7  # max number of iters to simulate in one graph update interval
    x_tick_interval = 1         # spacing between ticks (in hours)
    
    fig_width  = 7  # inches
    fig_height = 9 # inches
cfg = SimConfig()

In [ ]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.figure import Figure
import tkinter as tk

class DOSimWindow():
    """ Sim Window handles common MPL functions & data storage 
    as well as non-MPL tkinter configuration 
    """
    def __init__(self, frame, sim_config, events):
        
        # Unpack config items
        self.cfg = sim_config
        self.update_interval = cfg.update_interval
        self.xwindow = cfg.xwindow_hrs
        
        # MPL objects
        self.fig = Figure(figsize=(self.cfg.fig_width, self.cfg.fig_height))
        ax1 = self.fig.add_subplot(411)
        ax2 = self.fig.add_subplot(412)
        ax3 = self.fig.add_subplot(413)
        ax4 = self.fig.add_subplot(414)
        
        self.xdata = Data(self.xwindow*3600)
        self.pv_plot = PVPlot(ax1, self.xdata, self.xwindow*3600)
        self.gases_plot = GasesPlot(ax2, self.xdata, self.xwindow*3600)
        self.n2_pid_plot = PIDPlot(ax3, self.xdata, self.xwindow*3600)
        self.o2_pid_plot = PIDPlot(ax4, self.xdata, self.xwindow*3600)
        
        x_spacing = self.cfg.time_unit / (self.cfg.x_tick_interval * 3600)
        self.pv_plot.set_tick_interval(x_spacing)
        self.gases_plot.set_tick_interval(x_spacing)
        self.n2_pid_plot.set_tick_interval(x_spacing)
        self.o2_pid_plot.set_tick_interval(x_spacing)

        # misc
        self.pv_plot.axhline(self.cfg.simops.set_point)
        
        # Tkinter setup
        self.root = frame
        
        self.fig_frame = tk.Frame(self.root)
        
        # a tk.DrawingArea
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.fig_frame)
        self.canvas.show()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

        self.toolbar = NavigationToolbar2TkAgg(self.canvas, self.root)
        self.toolbar.update()
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)
        
        self._running = False
        self._monitor_id = None
        self._monitor_interval = 500
        
        # coroutine support
        self.do_coro = None
        self._current_state = {}
        
        # Periodic update support
        self.updating = True
        
        # Event handling for callbacks
        # All event handling works by modifying
        # Commands are sent directly to the coroutine
        # via use of command codes. 
        # The primary code is (obviously) used to run
        # the simulation. All the codes used below 
        # signal the coroutine to take specific action,
        # including modifying stateful values
        
        self.events = events
        
        def register_update_state(event, arg):
            def on_modified(ev, value, *args):
                self.trigger_update_state(arg, value)
                print("State Value Updated: %s %s " % (arg, value))
            self.events.register(event, on_modified)
                
        register_update_state("SP_CHANGED", "sp")
        register_update_state("PV_CHANGED", "pv")
        register_update_state("MG_CHANGED", "mg")
        
        def set_target_line(event, value):
            self.pv_plot.axhline(value)
        self.events.register("SP_CHANGED", set_target_line)

        
        def on_pid_changed(ev, value, *args):
            ev = ev.lower()
            ctrl, param, _ = ev.split("_")
            ctrl += "_pid"
            print("Value Changed: %s.%s: %.2f" % (ctrl, param, value))
            self.trigger_update_value(ctrl, param, value)
            
        for c in "N2", "O2":
            for p in "PGAIN", "ITIME", "DTIME", "BETA", "MODE", "MAN":
                ev = "%s_%s_CHANGED" % (c,p)
                self.events.register(ev, on_pid_changed)
        
        def on_process_gain_changed(ev, value, *args):
            c = ev.split("_")[0].lower()
            if c == "kmult":
                c = "k_mult"
            self.trigger_update_value("process", c, value)
            print("Changed value: %s to %s" % (c, value))
            
        for c in "K", "KMULT", "C", "DC", "D2C", "DELAY":
            ev = c + "_CHANGED"
            self.events.register(ev, on_process_gain_changed)
            
        def on_timefactor_changed(ev, value, *args):
            self.cfg.time_factor = value
            print(ev, ":", value)
        self.events.register("TIMEFACTOR_CHANGED", on_timefactor_changed)
        
        def on_clear_all_data(ev, *args):
            self.xdata.clear()
            for p in self.iter_plots():
                p.clear()
            self.do_coro.send(("SET_TIME", 0))
        self.events.register("CLEAR_ALL_DATA", on_clear_all_data)
        
        def on_advance_sim(ev, value, *args):
            if not value: return
            try:
                value = int(value)
            except ValueError:
                print("Invalid value: %r" % value)
                return
            print("Advancing simulation %d seconds..." % value)
            self._run_sim(value)
        self.events.register("ADVANCE_SIM", on_advance_sim)
        
        def on_legend_clicked(event):
            ll = event.artist
            series = self.leg_map[ll]
            series.toggle_visible()
            
        def on_xwindow_changed(ev, value, *args):
            nv = int(value * 3600)
            for p in self.iter_plots():
                for s in p.series:
                    if len(s) != nv:
                        s.resize(nv)
            if len(self.xdata) != nv:
                self.xdata.resize(nv)
        self.events.register("XWINDOW_CHANGED", on_xwindow_changed)
            
        # legend picker support
        self.leg_map = {}
        for p in self.iter_plots():
            m = p.get_leg_map()
            for leg, s in m.items():
                leg.set_picker(5)
                self.leg_map[leg] = s
        self.fig.canvas.mpl_connect("pick_event", on_legend_clicked)

        
    def trigger_update_value(self, ob, name, value):
        cmd = "UPDATE_VALUE"
        args = (ob, name, value)
        self.do_coro.send((cmd, args))
        
    def trigger_update_state(self, name, value):
        cmd = "MODIFY_STATE"
        args = (name, value)
        self.do_coro.send((cmd, args))
        
    def iter_plots(self):
        return (self.pv_plot, self.gases_plot, self.n2_pid_plot, self.o2_pid_plot)
        
    def pack(self, side=tk.TOP, fill=tk.BOTH, expand=1):
        self.fig_frame.pack(side=side, fill=fill, expand=expand)
        
    def start(self):
        self._updating = True
        self.begin_sim()
        self.schedule_update()
        
    def stop(self):
        self._updating = False
        
    def resume(self):
        if not self.do_coro:
            self.start()
        else:
            self.schedule_update()
        
    def begin_sim(self):
        self.do_coro = do_sim_coroutine(self.cfg.simops, self._current_state,
                         self.xdata, self.pv_plot.pv, self.pv_plot.c, self.gases_plot.co2,
                         self.gases_plot.n2, self.gases_plot.o2, self.gases_plot.air,
                         self.n2_pid_plot.uk, self.n2_pid_plot.up, self.n2_pid_plot.ui, self.n2_pid_plot.ud,
                         self.o2_pid_plot.uk, self.o2_pid_plot.up, self.o2_pid_plot.ui, self.o2_pid_plot.ud)
        next(self.do_coro)
        
    def update(self):
        n = self.cfg.time_factor
        n = int(n)
        self._run_sim(n)
        self.events.fire("PROCESS_STATE_UPDATE", self._current_state.copy())
        
    def _run_sim(self, n):
        self.do_coro.send(("SIM_ITERS", n))
        
        self.xdata.push(n)
        self.pv_plot.push(n)
        self.gases_plot.push(n)
        self.o2_pid_plot.push(n)
        self.n2_pid_plot.push(n)
        
        self.pv_plot.update()
        self.gases_plot.update()
        self.n2_pid_plot.update()
        self.o2_pid_plot.update()
        
        self.fig.canvas.draw()
        
    def schedule_update(self):
        def update(self=self):
            if self.updating:
                self.update()
                self.root.after(self.update_interval, update)
        self.root.after(self.update_interval, update)    
    


In [ ]:
from hello.pid.ui import *

class StatusDisplay(LabelDisplayFrame):
    def __init__(self, master, text, events, cfg):
        super().__init__(master, text, events)

        # text config....
        c,n,o = cfg.simops.initial_request_cno
        a = 1-(c+n+o)
        sub2 = "\u2082:"
        co2 = "CO"+sub2
        n2 = "N"+sub2
        o2 = "O"+sub2
        
        self.add_label("PV", "PV_PROCESS_UPDATE", "PV:", cfg.simops.initial_pv, 3, 6)
        self.add_label("CO2", "CO2_PROCESS_UPDATE", co2, "%.1f%%" % (c*100), 3, 6)
        self.add_label("N2", "N2_PROCESS_UPDATE", n2, "%.1f%%" % (n*100), 3, 6)
        self.add_label("O2", "O2_PROCESS_UPDATE", o2, "%.1f%%" % (o*100), 3, 6)
        self.add_label("Air", "AIR_PROCESS_UPDATE", "Air:", "%.1f%%" % (a*100), 3, 6)
        
        self.add_label("c", "C_PROCESS_UPDATE", "c:", cfg.simops.c, 3, 6)
        self.add_label("dc", "DC_PROCESS_UPDATE", "dc:", cfg.simops.dc, 3, 6)
        self.add_label("d2c", "D2C_PROCESS_UPDATE", "d\u00B2c:", cfg.simops.d2c, 3, 6)
        
        self.events.register("PROCESS_STATE_UPDATE", self.on_state_update)
        
    def on_state_update(self, ev, state, *args):
        self.labels['c'].set("%.4g" % state['c'])
        self.labels['dc'].set("%.4g" % state['dc'])
        self.labels['d2c'].set("%.4g" % state['d2c'])
        
        self.labels['PV'].set("%.2f%%" % state['pv'])
        self.labels["CO2"].set("%.1f%%" % (state['co2_req']*100))
        self.labels["N2"].set("%.1f%%" % (state['n2_req']*100))
        self.labels["O2"].set("%.1f%%" % (state['o2_req']*100))
        self.labels["Air"].set("%.1f%%" % (state['air_req']*100))
        
class PIDFrame(LabelFrameWithLabelEntries):
    def __init__(self, master, text, events, prefix, pid_ops):
        super().__init__(master, text, events)
        
        self.add_entry("PGain:", pid_ops.p, prefix + "_PGAIN_CHANGED")
        self.add_entry("ITime:", pid_ops.i, prefix + "_ITIME_CHANGED")
        self.add_entry("DTime:", pid_ops.d, prefix + "_DTIME_CHANGED")
        self.add_entry("Beta:", pid_ops.beta, prefix + "_BETA_CHANGED")
        self.add_entry("Man:", pid_ops.man_request, prefix + "_MAN_CHANGED")
        
        # Modes
        modes = [
            ("Auto", PIDController.AUTO),
            ("Man", PIDController.MAN),
            ("Off", PIDController.OFF)
        ]
        mode_order = [m[0] for m in modes]
        map = dict(modes)
        self.menu = MenuLabel(self, "Mode:", events, prefix + "_MODE_CHANGED", mode_order, map=map)
        self.menu.grid(row=len(self.entries), column=0, sticky=(tk.E,tk.W,tk.N,tk.S))
        

In [ ]:

class PIDSimFrame():
    def __init__(self, master, WindowClass, sim_cfg, events=None):
        if events is None:
            events = EventDispatcher()
        self.cfg = sim_cfg
        ops = cfg.simops
        self.events = events
        self.frame = tk.Frame(master)
        self.w = WindowClass(self.frame, sim_cfg, events)
        self.w.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
        self.settings_frame = sf = tk.Frame(self.frame)
        self.settings_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=1)
        
        # Widgets
        
        # N2 PID
        n2pid = PIDFrame(sf, "N\u2082 PID:", events, "N2", ops.n2_pid)

        # O2 PID
        o2pid = PIDFrame(sf, "O\u2082 PID:", events, "O2", ops.o2_pid)  
        
        # Process values
        proc = LabelFrameWithLabelEntries(sf, "Process:", events)
        proc.add_entry("Set Point:", ops.set_point, "SP_CHANGED")
        proc.add_entry("PV:", ops.initial_pv, "PV_CHANGED")
        proc.add_entry("k:", ops.k, "K_CHANGED")
        proc.add_entry("k_mult:", ops.k_mult, "KMULT_CHANGED")
        proc.add_entry("delay:", s2m(ops.delay), "DELAY_CHANGED")
        
        # Simulation options
        sim = LabelFrameWithLabelEntries(sf, "Simulation:", events)
        sim.add_entry("time factor:", cfg.time_factor, "TIMEFACTOR_CHANGED")
        sim.add_entry("x window(hr):", cfg.xwindow_hrs, "XWINDOW_CHANGED")
        
        # buttons
        clear = ttk.Button(sf, text="Clear", command=events.create_callback("CLEAR_ALL_DATA"))
        advance = EntryButton(sf, "Advance(s)", events, "ADVANCE_SIM")

        # Consumption constants
        c_frame = LabelFrameWithLabelEntries(sf, "Consumption:", events)
        c_frame.add_entry("c", ops.c, "C_CHANGED")
        c_frame.add_entry("dc/dt", 0, "DC_CHANGED")
        c_frame.add_entry("d\u00B2c/dt\u00B2", 0, "D2C_CHANGED")
        c_frame.set_trigger_mode("always")
        
        # Output display
        cstate = StatusDisplay(sf, "Current:", events, self.cfg)    
        
        tk_all = (tk.E, tk.W, tk.N, tk.S)
        n2pid.grid(row=1, column=1, sticky=tk_all)
        o2pid.grid(row=1, column=2, sticky=tk_all)
        
        proc.grid(row=2, column=1, sticky=tk_all)
        sim.grid(row=2, column=2, sticky=tk_all)

        c_frame.grid(row=3, column=1, rowspan=3, sticky=(tk.E, tk.W, tk.N, tk.S))
        cstate.grid(row=3, column=2, rowspan=8, sticky=(tk.E, tk.W, tk.N, tk.S))
        
        advance.grid(row=11, column=1)
        clear.grid(row=11, column=2, sticky=tk.S)
        
        self.wl = {}
        self.wl.update((k,v) for k,v in locals().items() if isinstance(v, tk.Widget))
        
        self.w.start()
        master.register_destroy_callback(self.w.stop)
        
        self.n2pid = n2pid
        self.o2pid = o2pid
        
        
    def pack(self, **kw):
        self.frame.pack(**kw)
        
        

In [ ]:
def run2():
    global root, f
    cfg.simops.o2_pid.p = 7
    cfg.simops.o2_pid.i = 50
    cfg.simops.o2_pid.amax = 100
    cfg.simops.o2_pid.deadband = 0
    cfg.simops.o2_pid.beta = 0
    cfg.simops.o2_pid.man_request = 10

    cfg.simops.n2_pid.p = -2
    cfg.simops.n2_pid.i = 60
    cfg.simops.n2_pid.amax = 90
    cfg.simops.n2_pid.deadband = 0
    cfg.simops.n2_pid.beta = 0
    cfg.simops.n2_pid.man_request = 10
    cfg.simops.delay=0*minutes

    cfg.update_interval = 50
    cfg.time_factor = 200

    cfg.simops.initial_pv = 100
    cfg.simops.set_point = 100

    cfg.xwindow_hrs = 10

    root = TkQuitHack()
    root.wm_title("DO Simulation")
    #root.geometry("%dx%d"%(600, 800))
    f = PIDSimFrame(root, DOSimWindow, cfg)
    f.pack()
    root.mainloop()
run2()

In [ ]:
#f.w._current_state['o2_pid'].b